# GenAI HW6: LLM Values Alignment
## Objectives
- Learn how to align a model's behavior using labelled preference data.

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to ntu-gen-ai-2024-spring-ta@googlegroups.com

## Install and import necessary libraries  (~2 min)
### Ignore the warning if the blockes finish successfully.

In [1]:
!pip install -q bitsandbytes==0.43.1 datasets==2.19.0 peft==0.10.0 trl==0.8.6 accelerate==0.29.3

^C
ERROR: Operation cancelled by user


## Load dataset

In [4]:
!git clone https://github.com/Baiiiiiiiiii/GenAI_hw6_dataset.git

fatal: 目标路径 'GenAI_hw6_dataset' 已经存在，并且不是一个空目录。


## Import Packages

In [1]:
import os
import torch
import re
import json
import gdown
from datasets import Dataset
import pandas as pd
from peft import LoraConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig, GenerationConfig
from tqdm.auto import tqdm
from trl import DPOTrainer

/home/scy/anaconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Open and load the json dataset
with open("./GenAI_hw6_dataset/labelled_data.json", 'r') as jsonfile:
    full_data = json.load(jsonfile)

with open("./GenAI_hw6_dataset/test_prompt.json", 'r') as jsonfile:
    test_data = json.load(jsonfile)

## Load model

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    'MediaTek-Research/Breeze-7B-Instruct-v0_1',
    device_map='auto',
    trust_remote_code=True,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4'
    )
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.42s/it]


## Get response from the original model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('MediaTek-Research/Breeze-7B-Instruct-v0_1')
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token

def data_formulate(data):
    messages = [
        {"role": "system", "content": '回覆請少於20字'},
        {"role": "user", "content": data['prompt']},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

original_model_response = []
for data in tqdm(test_data):
    id = data['id']
    print(f'Question {id}:\n'+data['prompt'])
    inputs = tokenizer(data_formulate(data), return_tensors="pt").to('cuda')
    generation_config=GenerationConfig(
            do_sample=False,
            max_new_tokens = 200,
            pad_token_id = tokenizer.pad_token_id
    )
    output = model.generate(**inputs, generation_config=generation_config)
    output = tokenizer.batch_decode(output, skip_special_tokens=True)[0].split('[/INST] ')[1]
    original_model_response.append(output)
    print('Response from original model:\n'+output+'\n')

  0%|          | 0/10 [00:00<?, ?it/s]

Question 1:
真人化是否能改善日本漫畫的全球可及性？


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
 10%|█         | 1/10 [00:03<00:30,  3.40s/it]

Response from original model:
真人化可能會提高日本漫畫的全球可及性，因真人版電影或劇集可以吸引更多非漫畫讀者的注意，並提供不同的體驗。然而，這取決於真人化作品的品質、行銷策略和市場接受度。

Question 2:
真人化如何影響年輕一代對日本漫畫的看法？


 20%|██        | 2/10 [00:06<00:24,  3.07s/it]

...

 30%|███       | 3/10 [00:09<00:22,  3.14s/it]

Response from original model:
真人化可能會提升原作漫畫的知名度和影響力，但文學價值本身可能因個人喜好和文化差異而異。真人化可能帶來更多觀眾，但文學價值取決於原作的故事、人物和主題，而非真人化形式。

Question 4:
真人化是否有助於保護和保存日本漫畫的傳統？


 40%|████      | 4/10 [00:11<00:16,  2.81s/it]

Response from original model:
真人化可能有助於提高日本漫畫的知名度和吸引更多觀眾，但是否真正保護和保存傳統尚需視真人化作品是否尊重原作精神和文化價值。

Question 5:
真人化是否有助於提升日本漫畫行業的經濟效益？


 50%|█████     | 5/10 [00:14<00:14,  2.81s/it]

## Set parameters
### You only need to modify this block. Please don’t alter any other parts.

In [ ]:
num_epoch = 1  # 1
data_size = 50
support_ratio = 1

## Prepare training data

In [ ]:
# Select part of the data for training
training_data = full_data[:data_size]

# Define the size of the support dataset
support_data_size = int(data_size * support_ratio)

# Prepare the data for the training dataset
prompt_list = [data_formulate(data) for data in training_data]
chosen_list = [data['support'] for data in training_data[:support_data_size]] + [data['oppose'] for data in training_data[support_data_size:]]
rejected_list = [data['oppose'] for data in training_data[:support_data_size]] + [data['support'] for data in training_data[support_data_size:]]
position_list = ['support' for _ in range(support_data_size)] + ['oppose' for _ in range(data_size - support_data_size)]

# Create the training dataset
train_dataset = Dataset.from_dict({'prompt': prompt_list, 'position': position_list, 'chosen': chosen_list, 'rejected': rejected_list})
pd.DataFrame(train_dataset).rename(columns={"chosen": "preferred", "rejected": "non-preferred"})

,prompt,position,preferred,non-preferred
0,<s>回覆請少於20字 [INST] 日本動漫真人化是否有損原作形象？ [/INST],support,真人化能夠呈現更真實的角色形象，提升原作魅力。,真人化可能無法完美呈現動畫中的獨特風格，損害原作形象。
1,<s>回覆請少於20字 [INST] 真人化是否能夠擴大動漫在全球的影響力？ [/INST],support,真人化能夠讓更多非動漫迷接觸作品，擴大影響力。,真人化可能失去動漫的獨特風格，限制影響力擴大。
2,<s>回覆請少於20字 [INST] 真人化是否能夠吸引新觀眾？ [/INST],support,真人化能夠吸引不熟悉動漫的觀眾，擴大受眾。,真人化可能讓原本的動漫迷感到失望，無法吸引新觀眾。
3,<s>回覆請少於20字 [INST] 真人化是否能夠保留原作故事情節的精髓？ [/INST],support,真人化有機會更深入挖掘原作故事，保留精髓。,真人化可能因為改編而失去原作故事的深度與精髓。
4,<s>回覆請少於20字 [INST] 真人化是否能夠提升動漫產業的商業價值？ [/INST],support,真人化能夠開拓更多商業機會，提升產業價值。,真人化可能讓觀眾對原作失去興趣，影響產業價值。
5,<s>回覆請少於20字 [INST] 真人化是否能夠保持原作的文化特色？ [/INST],support,真人化可以透過場景、服裝等元素保留文化特色。,真人化可能因為文化差異而失去原作獨有的文化魅力。
6,<s>回覆請少於20字 [INST] 真人化是否能夠挑戰技術上的新突破？ [/INST],support,真人化促使技術創新，挑戰視覺效果上的新高度。,真人化可能因為技術限制而無法達到動畫中的視覺效果。
7,<s>回覆請少於20字 [INST] 真人化是否會受到演員選擇的爭議？ [/INST],support,演員選擇可因應市場需求，不必受限於動畫形象。,演員選擇可能引起爭議，觀眾難以接受角色塑造。
8,<s>回覆請少於20字 [INST] 真人化是否能夠提高動漫的社會認同度？ [/INST],support,真人化有機會讓更多人接受動漫，提高社會認同度。,真人化可能因為劇情改編而無法贏得社會認同。
9,<s>回覆請少於20字 [INST] 真人化是否能夠保留原作角色的個性特色？ [/INST],support,真人化可以透過演員表現保留角色的個性特色。,真人化可能因演員演技或導演選擇而失去角色的原有特色。


## Training

In [ ]:
training_args = TrainingArguments(
    output_dir='./',
    per_device_train_batch_size=1,
    num_train_epochs=num_epoch,
    gradient_accumulation_steps=8,
    gradient_checkpointing=False,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps = 1,
    warmup_ratio = 0.1,
    report_to = 'none'
)

In [ ]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    beta=0.1,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
)

/home/scy/anaconda3/envs/llm/lib/python3.11/site-packages/trl/trainer/dpo_trainer.py:300: UserWarning: `max_length` is not set in the DPOTrainer's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/home/scy/anaconda3/envs/llm/lib/python3.11/site-packages/trl/trainer/dpo_trainer.py:307: UserWarning: `max_prompt_length` is not set in the DPOTrainer's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/home/scy/anaconda3/envs/llm/lib/python3.11/site-packages/trl/trainer/dpo_trainer.py:332: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
Map: 100%|██████████| 10/10 [00:00<00:00, 956.56 examples/s]


In [ ]:
dpo_trainer.train()

 33%|███▎      | 1/3 [00:03<00:06,  3.47s/it]

{'loss': 0.6931, 'grad_norm': 3.509688138961792, 'learning_rate': 0.0002, 'rewards/chosen': 0.0, 'rewards/rejected': 0.0, 'rewards/accuracies': 0.0, 'rewards/margins': 0.0, 'logps/rejected': -50.23879623413086, 'logps/chosen': -41.871788024902344, 'logits/rejected': -0.3050101697444916, 'logits/chosen': -0.38764747977256775, 'epoch': 0.8}


 67%|██████▋   | 2/3 [00:06<00:03,  3.41s/it]

{'loss': 0.6931, 'grad_norm': 3.7032129764556885, 'learning_rate': 0.0001, 'rewards/chosen': 0.0, 'rewards/rejected': 0.0, 'rewards/accuracies': 0.0, 'rewards/margins': 0.0, 'logps/rejected': -51.70563888549805, 'logps/chosen': -44.04267501831055, 'logits/rejected': -0.309154748916626, 'logits/chosen': -0.3647148013114929, 'epoch': 1.6}


100%|██████████| 3/3 [00:10<00:00,  3.42s/it]

{'loss': 0.3167, 'grad_norm': 2.78055477142334, 'learning_rate': 0.0, 'rewards/chosen': 0.3809282183647156, 'rewards/rejected': -0.7297425866127014, 'rewards/accuracies': 1.0, 'rewards/margins': 1.110670804977417, 'logps/rejected': -57.246177673339844, 'logps/chosen': -45.69913101196289, 'logits/rejected': -0.2910057306289673, 'logits/chosen': -0.3661591410636902, 'epoch': 2.4}


100%|██████████| 3/3 [00:11<00:00,  3.73s/it]

{'train_runtime': 11.1784, 'train_samples_per_second': 2.684, 'train_steps_per_second': 0.268, 'train_loss': 0.5676639676094055, 'epoch': 2.4}


TrainOutput(global_step=3, training_loss=0.5676639676094055, metrics={'train_runtime': 11.1784, 'train_samples_per_second': 2.684, 'train_steps_per_second': 0.268, 'total_flos': 0.0, 'train_loss': 0.5676639676094055, 'epoch': 2.4})

## Get response from the trained model

In [ ]:
trained_model_response = []
for data in tqdm(test_data):
    id = data['id']
    print(f'Question {id}:\n'+data['prompt'])
    inputs = tokenizer(data_formulate(data), return_tensors="pt").to('cuda')
    generation_config=GenerationConfig(
            do_sample=False,
            max_new_tokens = 200,
            pad_token_id = tokenizer.pad_token_id
    )
    output = model.generate(**inputs, generation_config=generation_config)
    output = tokenizer.batch_decode(output, skip_special_tokens=True)[0].split('[/INST] ')[1]
    trained_model_response.append(output)
    print('Response from trained model:\n'+output+'\n')

  0%|          | 0/10 [00:00<?, ?it/s]

Question 1:
真人化是否能改善日本漫畫的全球可及性？


 10%|█         | 1/10 [00:03<00:31,  3.55s/it]

Response from trained model:
真人化可能會提高日本漫畫的全球可及性，因真人版電影或劇集能吸引更多非漫畫讀者，並拓展不同文化的市場。然而，成功與否仍取決於故事內容、製作品質、行銷策略等因素。

Question 2:
真人化如何影響年輕一代對日本漫畫的看法？


 20%|██        | 2/10 [00:05<00:22,  2.82s/it]

...

Response from trained model:
真人化可以促進日本漫畫的創新和多樣性，因它能吸引更多觀眾，拓展市場，並提供不同的創作機會。然而，平衡和尊重原作者的想法是關鍵。

Question 9:
真人化是否有助於擴大動漫文化的市場份額？


 90%|█████████ | 9/10 [00:28<00:03,  3.11s/it]

Response from trained model:
真人化可以有助於擴大動漫文化的市場份額，因它能吸引更多非動漫迷觀眾，並透過不同媒介推廣動漫文化。然而，成功與否取決於作品品質、行銷策略和市場接受度。

Question 10:
真人化是否有助於提高日本漫畫在全球的競爭力？


100%|██████████| 10/10 [00:30<00:00,  3.00s/it]

Response from trained model:
真人化可以提高日本漫畫在全球的競爭力，因它能吸引更多觀眾，拓展市場，並促進文化交流。



## Please observe the output of this block to complete your report, and don't forget to take a screenshot of the results

In [ ]:
model_response = []
print(f'num_epoch: {num_epoch}\ndata_size: {data_size}\nsupport_ratio: {support_ratio}')
print()
for data in test_data:
    id = data['id']
    ref_output = original_model_response[id-1]
    output = trained_model_response[id-1]
    print(f'Question {id}:\n'+data['prompt'])
    print('Response from original model:\n'+ref_output)
    print('Response from trained model:\n'+output)
    print()
    model_response.append({'id':data['id'], 'prompt':data['prompt'], 'response_from_original_model':ref_output, 'response_from_trained_model':output})

num_epoch: 3
data_size: 10
support_ratio: 1

Question 1:
真人化是否能改善日本漫畫的全球可及性？
Response from original model:
真人化可能會提高日本漫畫的全球可及性，因真人版電影或劇集可以吸引更多非漫畫讀者的注意，並提供不同的體驗。然而，這取決於真人化作品的品質、行銷策略和市場接受度。
Response from trained model:
真人化可能會提高日本漫畫的全球可及性，因真人版電影或劇集能吸引更多非漫畫讀者，並拓展不同文化的市場。然而，成功與否仍取決於故事內容、製作品質、行銷策略等因素。

Question 2:
真人化如何影響年輕一代對日本漫畫的看法？
Response from original model:
真人化可能會影響年輕一代對日本漫畫的看法，使他們更容易接受和理解故事和角色，並吸引更多人關注和支持日本漫畫文化。然而，個人喜好和文化差異可能導致不同的影響。
Response from trained model:
真人化可以提高年輕一代對日本漫畫的關注度和接受性，但個人喜好和審美觀差異仍存在，影響程度因人而異。

Question 3:
真人化是否能提升原作漫畫的文學價值？
Response from original model:
真人化可能會提升原作漫畫的知名度和影響力，但文學價值本身可能因個人喜好和文化差異而異。真人化可能帶來更多觀眾，但文學價值取決於原作的故事、人物和主題，而非真人化形式。
Response from trained model:
真人化可能會提高原作漫畫的知名度和影響力，但文學價值本身並不會因此提升。文學價值取決於故事、人物、主題和寫作技巧，真人化只是一種呈現方式。

Question 4:
真人化是否有助於保護和保存日本漫畫的傳統？
Response from original model:
真人化可能有助於提高日本漫畫的知名度和吸引更多觀眾，但是否真正保護和保存傳統尚需視真人化作品是否尊重原作精神和文化價值。
Response from trained model:
真人化可以提高日本漫畫的知名度和影響力，但保護和保存傳統上，以動畫和漫畫的形式更符合文化傳承。真人化可能帶來商業利益，但傳統的漫畫形式更貼近日本漫畫的本質。

Qu

## Get the output file

In [ ]:
with open(f"epoch-{num_epoch}_size-{data_size}_ratio-{support_ratio}.json", "w", encoding='UTF-8') as outfile:
    json.dump(model_response, outfile, indent=4, ensure_ascii=False)